## Itaconic acid production

Andre initiated some clever ideas on the effect of limited oxygen supply on the production of Itaconic acid. I want to set up a Ipython notebook basis for tackling the problem. Let's start with the metabolism from glucose.

The full metabolic map will be given by the following:

<img src="Ita_full.png" width="650" />

It is easier to simplify the map (and flux model) to the following:

<img src="Ita_model.png" width="850" />

Let's define some arbitrary physiological parameters:


|$\alpha \qquad \qquad$|$\gamma \qquad \qquad $|$\mu_{max} \qquad$|$\theta_{max}\qquad$| $(P/O)$ $\qquad \qquad$
|----|-|-|-|-
|$\frac{cmol\,CO_2}{cmol \, X}$|$\frac{mol \, ATP}{Cmol \, X}$|$\frac{1}{h}$|$\frac{mol \, ATP}{cmol\,X\cdot h}$|$\frac{mol\,ATP}{mol\, O}$
|0.1|2.5|0.15|0.1|1.5

Let's get Python fired up:


In [5]:
import numpy
import matplotlib.pyplot as plt
%matplotlib notebook 
from scipy.integrate import odeint
alpha, gamma, mu, theta, PO = 0.1, 2.5, 0.15, 0.1, 1.5

The basis of the stoichiometric matrix (Sbase) will be the two nodes, NADH and ATP balance:

In [6]:
Sbase = numpy.matrix([[-1,1.1,1,1,0,0,0],
                     [0,0,0,-1,1.5,1.2,0],
                     [0,0.1,2,1/3,-0.5,0.2,-2],
                     [0,-2.5,0.5,1/3,0,-0.2,3]])

Note that the 0.5 mol ATP on flux 2 is not $\frac{2}{3}$ like always to compensate for the FADH that will be considered as NADH.
Given 7 rates, 3 specifications is required. Let's define the possible spec rows that will be added to Sbase.

In [29]:
et_spec=[0,0,0,0,1,0,0]
ox_spec=[0,0,0,0,0,0,1]
res_spec=[0,0,1,0,0,0,0]
ita_spec=[0,0,0,0,0,1,0]
mu_spec=[0,1,0,0,0,0,0]
s_spec=[1,0,0,0,0,0,0]

We can now construct a 7x7 matrix of our choice for example:


In [30]:
S = numpy.vstack([Sbase,et_spec,res_spec,mu_spec])  #Stoich matrix with specs included
      
C= numpy.matrix([[0,0,0,theta, 0, 0, mu]]).T    #zero ethanol and respiration
r1 = numpy.linalg.solve(S,C) 
print(r1)

[[  6.58636364e-01]
 [  1.50000000e-01]
 [ -1.33366665e-17]
 [  4.93636364e-01]
 [  0.00000000e+00]
 [  4.11363636e-01]
 [  1.30909091e-01]]


So above is the rates (in $\frac{cmol}{cmol \, X \cdot h}$) when zero TCA takes place and when zero ethanol is excreted. One can just devide by $-r_S$ to get the yields:

In [31]:
Y=r/r[0]
print(Y)

[[ 1.        ]
 [ 0.50970874]
 [ 0.43932039]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.46480583]]


Let's 'switch off' itaconic instead of respiration and compare:

In [18]:
S = numpy.vstack([Sbase,et_spec,ita_spec,mu_spec])  #Stoich matrix with specs included
      
C= numpy.matrix([[0,0,0,theta, 0, 0, mu]]).T    #zero ethanol and respiration
r2 = numpy.linalg.solve(S,C) 
print(r2)

[[ 0.29428571]
 [ 0.15      ]
 [ 0.12928571]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.13678571]]


In [21]:
print([r2[6]/r1[6],r2[0]/r1[0]])

[matrix([[ 1.04489087]]), matrix([[ 0.44681061]])]


So 4.4% more oxygen is required to 'service' the energy requirements when using respiration. On the other hand less than half of the glucose is required when using respiration instead of itaconic acid production. Thanks for noticing this Andre!

Andre also showed that there are more energy efficient routes to produce itaconic acid. Both these routes will cause the ATP contribution to flux 5 (currently $\frac{1}{5}$ mol ATP) to be zero. Lets atler the energy balance and re-run the calculation: 

In [22]:
Sbase[3,5]=0  #altering the energy balance

S = numpy.vstack([Sbase,et_spec,res_spec,mu_spec])  #Stoich matrix with specs included
C = numpy.matrix([[0,0,0,theta, 0, 0, mu]]).T    #zero ethanol and respiration
r1 = numpy.linalg.solve(S,C)

S = numpy.vstack([Sbase,et_spec,ita_spec,mu_spec])  #Stoich matrix with specs included
C = numpy.matrix([[0,0,0,theta, 0, 0, mu]]).T    #zero ethanol and respiration
r2 = numpy.linalg.solve(S,C)

print([r2[6]/r1[6],r2[0]/r1[0]])

[matrix([[ 1.2221404]]), matrix([[ 0.50504479]])]


If the pathway exists, the oxygen requirements is even less. 

Remember from CBI that the oxygen supply per cell will get less if the biomass grows beyond the point where the volumetric mass transfer becomes limiting. [Click here if you forgot](https://nbviewer.jupyter.org/github/willienicol/Biochemical-engineering-notes/blob/master/6%20Modeling%20stoichiometry%20variations/Oxygen%20starvation%20in%20batch%20fermenters.ipynb).



So the idea is that old _Aspergillus terreus_ will gradually change from using respiration for generating energy, to using itaconic acid production as the oxygen supply becomes an issue. The question is how will the bug transition? Will $-r_S$ actually increase around this critical point or is the maximum glucose uptake rate ($\lambda$) already utilised by the respiration mode (when oxygen is freely avaialble). We don't know the answer yet, but Deon is on it.

If $\lambda$ was already used during respiration, a regime will form (as $r_O$ is decreased, remember $r_O$ is per cell) where both respiration and itaconic production will occur, while $\lambda$ is maintained. This regime will end when respiration becomes zero. For this regime we will specify $\lambda$. 

Before we begin let's reset the energy balance to the original:

In [24]:
Sbase[3,5]=-1/5  #altering the energy balance

S = numpy.vstack([Sbase,et_spec,res_spec,mu_spec])  #Stoich matrix with specs included
C = numpy.matrix([[0,0,0,theta, 0, 0, mu]]).T    #zero ethanol and respiration
r1 = numpy.linalg.solve(S,C)

S = numpy.vstack([Sbase,et_spec,ita_spec,mu_spec])  #Stoich matrix with specs included
C = numpy.matrix([[0,0,0,theta, 0, 0, mu]]).T    #zero ethanol and respiration
r2 = numpy.linalg.solve(S,C)

print([r2[6]/r1[6],r2[0]/r1[0]])

[matrix([[ 1.04489087]]), matrix([[ 0.44681061]])]


$\lambda$ can be determined form $r_2$. Let see by how much $r_O$ have to decrease to end up with full itaconic production:

In [27]:
lam=r2[0]

S = numpy.vstack([Sbase,et_spec,res_spec,s_spec])  #Stoich matrix with specs included
C = numpy.matrix([[0,0,0,theta, 0, 0, lam]]).T    #zero ethanol and respiration
r3 = numpy.linalg.solve(S,C)

print([r3[6]/r2[6]])

[matrix([[ 0.45470978]])]


Wow, so the inbetween regime (respiration and itaconic acid production) will last for a significant time, the oxygen will have to be decreased by more than a factor of 2. Let's see how the growth rate drops over this regime:

In [28]:
print([r3[1]/r2[1]])

[matrix([[ 0.33699634]])]


Significant!


Beyond this regime, one will have to use a $r_O$ specification in conjucntion with an itaconic specification. Both $\mu$ and $-r_S$ will now drop.

Note that a growth limitation was not considered in the calculation!

So there you have it, an initial approach to understanding itaconic acid production.